## Introduction

It dawned on me that this is why I thought of the aptly named notebooks when I want to explore new things. In short, I'm lookng at other's code and seeing what I can garner.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head()

(76518, 38)


,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

df.columns #Phew, found the original website.

Index(['id', 'marital_status', 'application_mode', 'application_order',
       'course', 'daytime/evening_attendance', 'previous_qualification',
       'previous_qualification_(grade)', 'nacionality',
       'mother's_qualification', 'father's_qualification',
       'mother's_occupation', 'father's_occupation', 'admission_grade',
       'displaced', 'educational_special_needs', 'debtor',
       'tuition_fees_up_to_date', 'gender', 'scholarship_holder',
       'age_at_enrollment', 'international',
       'curricular_units_1st_sem_(credited)',
       'curricular_units_1st_sem_(enrolled)',
       'curricular_units_1st_sem_(evaluations)',
       'curricular_units_1st_sem_(approved)',
       'curricular_units_1st_sem_(grade)',
       'curricular_units_1st_sem_(without_evaluations)',
       'curricular_units_2nd_sem_(credited)',
       'curricular_units_2nd_sem_(enrolled)',
       'curricular_units_2nd_sem_(evaluations)',
       'curricular_units_2nd_sem_(approved)',
       'curricular_units

### The First New Thing(s) & The Admittance of Mistakes

Also, copying relevant code to run whatever:

Notebook's way over the top for me. Even if it'd be meant for a non-technical audience.... way too many colors and stuff.

Tangentially, when there's no formal feature engineering a bit odd tog o so in depth about the features.

In [8]:
print(df.select_dtypes(include=int).columns.to_list()[:5])
categorical_features = df.select_dtypes(include=int).columns.to_list()
continuous_features = df.select_dtypes(include=float).columns.to_list()

['id', 'marital_status', 'application_mode', 'application_order', 'course']


A very neat bit of code I just noticed. Yes! This is what I was looking for to better sift through features, especially when often I've noticed with my eda the relevant analysis and code to look at, say int. vs. float are quite different.

In [17]:
#This bit is so tangential, but I just realized:

unique_categories = df['target'].unique()
# print(unique_categories)
category_to_int = {category: idx for idx, category in enumerate(unique_categories)}
df['target_encoded'] = df['target'].map(category_to_int)

print(df['educational_special_needs'].corr(df['target_encoded']))
print(df['curricular_units_1st_sem_(credited)'].corr(df['target_encoded']))
print(df['curricular_units_1st_sem_(without_evaluations)'].corr(df['target_encoded']))
print(df['curricular_units_2nd_sem_(credited)'].corr(df['target_encoded']))
print(df['curricular_units_2nd_sem_(without_evaluations)'].corr(df['target_encoded']))

0.0016064794739226625
-0.03339695005861841
0.05336534850191106
-0.03649945747047842
0.06881799497047925


Yeah, I really should have been a bit more rigorous with my checking eearlier before I delete matters. Ie I negelected to check if all the special education people graduated. So, albeit a trait that rarely occurs but if it does... Pragmatically, we ended up fine in that case and likely the others, but still...

In [11]:
#Another nifty bit of code:
discrete_features = [col for col in categorical_features if col.startswith("curricular")] + ["age_at_enrollment"]
print(len(discrete_features))
#I find so much elegance in this bit of code, directly adding one feature to whatever withstands the prior list compression

#Wait... these are the only discrete features?!?! Eh, not getting into that right now...

#Reminder that these are simply the names of the featues.

#It's much more graceful than:

discrete_features = [col for col in categorical_features if col.startswith("curricular")]
discrete_features.append("age_at_enrollment")
print(len(discrete_features))

#Ie the adding of lists together appends them.

11


Sigh, our host made a mistake so much of the actuarial PA exam attempted to prevent - nice graphs or print outs but explain what all these numbers mean without a flood...

Ie actually comment on the nature of some of them... What should we care about? Eveything?!!?

So too with his visuals on counts, distribution, 

Oh, and I took for granted the following:

With various features that are, bottom-line, represented by integers, such as course number, care likely should be taken to dummify those as they are not ordinal. Might as well experiment with that (see modeling_i).

In [ ]:
#Oh, nicehatmap color:

#cmap="coolwarm"

#Yeah, that will definiely make things so much easier to actually tell. Can't believe I didn't see something like
#ths earlier. Then it's actually a a heatmap...

Nice headings, summaries for each data sciency thing he's doing (such as PCA). Again, for this format I doubt it's neccesary to go into that much detail though besides just saying something like, "Reducing the amoun of features to make the data simpler via an established technique called PCA."

I do wonder the need for PCA when doing fancier techniques such as XGBboosting. Pragmtically, it's like, "Try both noob!", but even with our dummified data we have a mere 150ish for 75k entries (albeit before train/test split).

Likely I can glean a lot more from this, but I'll leave it at here. of Special note is that my current (and even my submission) XGBoost is higher than what this guy got. Quite interesting...

## Notebook Number 2

In [53]:
df.iloc[0, 0] = np.nan

#Kinda nice bit here, making it easier to notice the nulls:
df.isna().sum().sort_values(ascending=False)[:4] #Then add the list referencng to make it smaller...
#Sort ascending seems a bit overkill in retrospect as anything greater than 0 will by default be at the top...

id                                     1
curricular_units_2nd_sem_(credited)    0
international                          0
curricular_units_1st_sem_(credited)    0
dtype: int64

In [46]:
#Hmm, let's improve this:
nulls = {df.isna().sum().index[i]:df.isna().sum()[i] for i in range(len(df.isna().sum().sort_values(ascending=False))) if df.isna().sum()[i] > 0}
if len(nulls) > 0:
    print(f"And we have the following nulls:")
    print(nulls)
else:
    print(f"Phew, we have no nulls... this time...")
    
#Thanks to ChatGPT for telling me I have to use print when in the conditional, despite it being the last bit

#Likely all this is a bit overkill, but eh...

And we have the following nulls:
{'id': 1}


Unfortunately once again a deluge of information with little explanation. Ie I'll give them points for something when they wrote, "# View the statistical description of training dataset" before doing df.describe().T

In [58]:
df.describe().T[:4] #and shoutout re. the .T... much easier to go through this way. May perclude bothering to sort
#via categorical or not when things are this efficient.

,count,mean,std,min,25%,50%,75%,max
id,76517.0,38259.000000,22088.699611,1.0,19130.0,38259.0,57388.0,76517.0
marital_status,76518.0,1.111934,0.441669,1.0,1.0,1.0,1.0,6.0
application_mode,76518.0,16.054419,16.682337,1.0,1.0,17.0,39.0,53.0
application_order,76518.0,1.644410,1.229645,0.0,1.0,1.0,2.0,9.0


I like their attempted classification for better visualizations (categorical vs. numerical), which we just debates, yet their approach... assuming if categorical if only 8 or more...

cat_cols = [col for col in train.columns if train[col].nunique() <= 8]

Seems quite untrue... ie recall father's education to give one example? Rather, the approach would seemingly be if float or not. And, even then would still need some manual inspection (per my current knowledge) consideing potential features such as SAT scores. Off hand don't see any easy way to do that. I would like to have a convenient way to do that though...

--------
Some nice graphs, especially now that he bisected the features so the type of coding to show graphs for each one isn't overbearing.

Am I beinng a scoffer whe so much of the code is screaming ChatGPT to me? I personally don't mind, but at least cite it. Then again, I'm soooo curious as to the perceentage of people's workbooks that are pure ChatGPT and the like. I think it would expain the overall disapointment I feel when I look at the comments these people give to their work. Code wise obviously up the par.

----

Due earlier, but nice headings such as, "X.Y Whatever" then a nice underline underneath. Still a bit flashy, but not too much. I do think the underline is a bit much between subsections and they're short. Ie if we have a behemoth one after another...

Oh, and a real fancy bar inbetween sections. I do like that. Can then justify the line a lot more when it's even clearly the change from chapter/section to the next.

----

Shame with his models though; our 'noobish' XGBost beat his top scorer, CatBoost, by quite a bit. Furthermore, our simple logreg with simplified other categories did better than his gradient boosting model.

Yeah, no hyperparameter tuning at all...WHY?!?!?!!

--------

Nice bit of code with fature importance. A bit tempted to copy it. I think we can simplify it, as for sure I wouldn't bother doing it for every model.

Yeah, besides that it's just like why are you showing feature importance...At least with other stuff the lack of explanation can be tolerated. But, at the very end, what are you doing?!!??! Do you want a simpler model?? Do you want to do feature engineering with those?!!?

In [ ]:
## From a person at Kaggle, first obsered in Playground 4.6 Academic Success

# def plot_feature_importances(model, model_name, color_scale='Reds', dataframe=None):
#     """
#     Plots feature importances of a fitted random forest model.

#     Parameters:
#     model (RandomForest model): The trained random forest model.
#     color_scale (str): Color scale for the plot.
#     dataframe (pd.DataFrame): DataFrame used to train the model. Must not be None.

#     Returns:
#     Plotly Figure: A plot showing feature importances.
#     """
#     if dataframe is None:
#         raise ValueError("Dataframe cannot be None and must contain the feature names.")

#     # Extracting feature importances and sorting them
#     importances = model.feature_importances_
#     indices = np.argsort(importances)[::-1]
#     feature_names = dataframe.columns

#     # Creating a DataFrame for the importances
#     feature_importances = pd.DataFrame({
#         'Feature': feature_names[indices],
#         'Importance': importances[indices]
#     })

#     # Plotting the feature importances
#     fig = px.bar(feature_importances.sort_values('Importance', ascending=True), 
#                  x='Importance', 
#                  y='Feature',
#                  title=f"Feature Importances in {model_name}",
#                  labels={'Importance': 'Importance', 'Feature': 'Feature'},
#                  height=1400,
#                  color='Importance',
#                  color_continuous_scale=color_scale)

#     fig.update_layout(xaxis_title='Importance', yaxis_title='Feature')

#     return fig

# model_name = 'Random Forest'
# fig = plot_feature_importances(rf_model, model_name, 'Picnic', X_train)
# fig.show()

## Notebook 3

Phew, some redemption - finally a guy that writes what he's doing.

----------

Interesting idea re. "...for that much data we'll nto always do a five-fold cross-validation..." For sure if we're doing something more lengthy like SVMs, but I found that even running many many combinations on XGBoost to not take that long. Plus to be sure I still fail to see past the argument, "Why not?", considering the 'time save' to be negligible.

Hmm, a pie chart. Eh, guess when just 3 it's fine - plus he still had the percentages inside.

"Note that we cannot distinguish categorical features from integer features by looking at the data. We can only distinguish them by looking at the documentation!" What a boss! Yeah, perhaps we can't have a simplified approach as we dreamed of in our youth (of 15 minutes ago).

Shame he still did the barrage of distributions, but when he actually explains them!!! Plus, neat how he uses different colors for different types: integers, categories, and floats as blue, black, and green (bar, bar, histograms interestingly).

Also, shame he didn't use the mask. Maybe he's not into it though. Plus, "DISCLAIMER: It is very questionable whether one should compute correlations for categorical features, but we do it anyway."

Oh, it just dawned on me - what's his code?!!? Sigh, I'll take the codeless explanation over code without it at least this time. Shame, as I must admit I'd prefer the later anydays - as who cares how much of a smooth criminal we're dealng wth if we have no clue how he got there (by moonwalking, excuse me).

Shame too as he made the heatmap more readible by multiplyng by 10. A bit ironic considering the name of the visual though.

----

Interesting bit of comparing the top feature's distribution in the train/test and validatio data. Suppose it's worthy to do. Unsure how he used cirriculate units, which we do know to be the most significant. Perhaps via correlations; oh wait, that's pure unsupervised there - no target in sight...

Very nice useage of decision trees to highlight pragmatically why that feature is so significan.

----- 

Not up to it at the moment, but lkely worthwhie to make noe of his top model, ensemble (ridge) which was .002 greater than ours. Plus, what's these 'tuned' he keeps on mentioning? Furthermore, what's DART? Also, shoutout to him using extra trees.

----

Shoutout out again for him stating his conclusion. Besides eliminating useless, he wants to create additional.How though - do you mean via engineering or going into the field? Love how he goaded people to actually tune models. And, fun last oe, "Find a better ensembling method." Likely reserved for the phders. Might be there one day, you never know...

Overall I am quite pleased with his work, albit not adding any new code here; adding and cemeting my knowledge.


## Notebook 4

Gah, already way to many visuals and an about me seems over the top. Especially when it's the same over the top one after every section.

I will give them credit for some fancy codes, but these monster formulas/methods I'm never fond of. Recalling some of my early macros in my first internship...

A lot of fluff in my opinion, but I will also laud them for a nice display of unique values. Then again, we do have the data dictionary which is essentially that. Also, will laud the nice font manipulations and backgrounds.

------------

Hmm, they also wanted to try to classify features by category, with a threshold, default 50, for them. Better than the 8, but still... we honestly got close to that earlier...<br> Again, like I may have wrote earlier, perhaps something like that is worthwhile as a default, then spruce up the exceptions later, but it ultimately still requires human intervention.

Now, if we have objects instad of numbers, ie 'married' instead of 2, then perhaps a function/loop to log those and map them would be worthwhile. Could make that later.

------------

Sigh, a barrage of visuals again. Although I obviously have what to work on, even if I do print out every count, for example, I'll comment on what we're seeing... Once again a line or two of introduction - that's it...

-----

I feel like I've been too critical. Thankfully, we have something to praise (genuinely and again) - adressing outliers. Yes, I forgot to do that. Relevant for any numeric feature. Likely curbing or dropping this could also yield much greater accuracy. A shame, once again, though that there was no commentary at all.

----

Whaaaaat? By also using XGBoost .6, 400, 3, .29, col by smple .5 and a test accuracy of .852?!?!!? Must have ben some serious discrepancies between this and the final as the highest was barely over .84...

## Notebook 5

Fancy starter visual, but was actually nice (in my opinion) as i marks different students with different images representing different features all weaving togther.

---------

Boss, this guy is actually describing stuff. Well, at least at the beginning....

Oyyyyy. I didn't even consider the class-imbalance a potential issue. We stratified, but I assumed that when we still have a health 20ish percent in the minority we'd be fine.... Definitely worth playing with that.

Worth also noting that he, and truthfully others, also checked for duplicate data. Honestly should be doing that myself. Recall the first iteration of my reddit project?!?!

----

Woooh, he made observations about his visuals. Albeit I'd claim some are shallow, but still... we finally found a comrade!

Interesting plots where by each category, such as marital status, he directly compares the three targets. A bit overwhelming dependng upon the graph. And, ofte would need to be scaled or have their numbers on top, as many of them are overwhelming in one group and the rest are lilluputions (sp?).

Unsure what he's adding with the pie chart when we already say a bar of the targets eariler.

Noooooooooooooooo. He let me down with that color scheme. Albeit cool he made a custom one, but wha is that - blue to orange to green?!!?

-----

Of note is him keepig all the feeatures.

Hmm, for his log reg he specified multiclass = 'ovr'. Likely bad I did nothing, but eh, we did wel with it...

Is he note tuning his guys? Noooo.

Interesting visuals with his KNN - erro rate vs k value.

Also, I do like his classification report, but a shame he didn't introduce it. Furthermore, when ultimatly our final metric for Kaggle is accuracy... But still, from a tuning perspective could be the way to go about it, to get that higher accuracy, is by focusing perhaps on another metric.

----

Hmm, a bit interesting he's going over everything again with the holdout data re. analysis. Albeit good to check, which I took for granted I think, but to re-write everything by default...

## Notebook 6

Time for a final one before doing something else, with this project or another.

I'll comment on the first notebook I see where I add a comment of benefit to me, whether in code or organzation.

----

Well, I won't hesitate to complain about "EDA" being just df.describe().T without a single word of explanation - feh!

A number of interesting graphs, whether positive or negative, but for the sake of brevity I won't bother to comment. In short don't let your reader have epilispy (sp?) with all the colors.

Interesting to play around with select cubic+ engineered featues. Unsure if XGBoost would pick it up, but at the least gotta laud him for that. However, DON"T EVER ADD THEM per linear algebra that I don't honestly remember.

----------------

Guess we're done with these notebooks for now. Pragmatically I didn't try one of XGBoost's brothers, but I can't say I'm too motivated when ultimately it'll come down to, "I can't explain it mathematically, but it works!"

Now, I did notice a bit of people get higher accuracy via many different hyperparamters, even within XGBoost, but the pragmatics of doing that seem negligible, save if the data science problem statement (which technically would be valid for Kaggle competitions) justifies using something like that.

If I decide to do more competitions, which I'm debating, it does give easy github activity, to just focus on any peer notebook that isn't pounding XGBoost and the like, out of hope that they actually comment, do formal feature engineerng, etc.

Perhaps I need to just find a second 'capstone' to keep myself occupied with. We'll see!

In [56]:
df.head()

,id,marital_status,application_mode,application_order,course,daytime/evening_attendance,previous_qualification,previous_qualification_(grade),nacionality,mother's_qualification,...,curricular_units_2nd_sem_(enrolled),curricular_units_2nd_sem_(evaluations),curricular_units_2nd_sem_(approved),curricular_units_2nd_sem_(grade),curricular_units_2nd_sem_(without_evaluations),unemployment_rate,inflation_rate,gdp,target,target_encoded
0,0,1,1,1,9238,1,1,126.0,1,1,...,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate,0
1,1,1,17,1,9238,1,1,125.0,1,19,...,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout,1
2,2,1,17,2,9254,1,1,137.0,1,3,...,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout,1
3,3,1,1,3,9500,1,1,131.0,1,19,...,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled,2
4,4,1,1,2,9500,1,1,132.0,1,19,...,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate,0
